#### The Leverage Effect

In [ ]:
# Import packages
import pandas as pd 
import numpy as np
import yfinance as yf
import scipy.stats as sp
import matplotlib.pyplot as plt

In [ ]:
# import data
# bond = the bond of the country, stock = the stock/index being analysed
ticker = ["LMT"] # Here the desired stock/index is chosen ["LMT", "SHEL.L", "AMZN", "^GSPC", "^FTSE"]
start = "1990-01-01"
end = "2020-12-31"
df = pd.DataFrame(yf.download(ticker, start, end)["Adj Close"])
df = df.rename(columns = {"Adj Close": "Close"})
df = df.dropna()

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
#  create log returns of asset
df["log_rtn"] = np.log(df.Close / df.Close.shift(1))

df = df.dropna()

In [ ]:
#  create a daily and monthly volatility of the asset
df["moving_std_21"] = df[["log_rtn"]].rolling(window=21).std()
df["moving_std_252"] = df[["log_rtn"]].rolling(window=252).std()

df = df.dropna()

In [ ]:
# Plotting graph of the simple returns, log returns, and volatilities
import matplotlib.dates as mdates
plt.rcParams.update({"font.size": 21})

#  plot a 3 way plot for qualitative analysis
fig, ax = plt.subplots(3, 1, figsize=(25, 12), 
                       sharex=True)

df.Close.plot(ax = ax[0])
ax[0].set(title = list(ticker)[0] + " Time Series", # Automated changing of title name inline with RIC
          ylabel = "Price ($)")

df.log_rtn.plot(ax = ax[1])
ax[1].set(ylabel = "Log returns")

df.moving_std_21.plot(ax = ax[2], color = "g", label = "21d Vol")
df.moving_std_252.plot(ax = ax[2], color = "r", label = "252d Vol")
ax[2].set(ylabel = "Moving Volatility",
          xlabel = "Date")
ax[2].legend()
ax[2].xaxis.set_major_locator(mdates.YearLocator(2))

#plt.tight_layout()
#plt.savefig(f"{ticker}.png")

plt.show()

In [ ]:
# select a window of data to test correlation for
win = df["1990":"2020"]

# calculate correlations
# asset to 21d volatility
p21 = sp.pearsonr(win.Close, win.moving_std_21)
s21 = sp.spearmanr(win.Close, win.moving_std_21)
df_21 = [p21[0], p21[1], s21[0], s21[1]]

# asset to 252d volatility
p252 = sp.pearsonr(win.Close, win.moving_std_252)
s252 = sp.spearmanr(win.Close, win.moving_std_252)
df_252 = [p252[0], p252[1], s252[0], s252[1]]

# present the list as a data frame
data = {"Asset/21d": df_21, "Asset/252d": df_252}
corr_df = pd.DataFrame(data, index = ["Pearson", "p-value", "Spearman", "p-value"])
corr_df.round(3)

In [ ]:
# vix to s&p500 correlation
# import data
# bond = the bond of the country, stock = the stock/index being analysed
ticker = ["^GSPC", "^VIX"]
start = "1990-01-01"
end = "2020-12-31"
vix = pd.DataFrame(yf.download(ticker, start, end)["Adj Close"])
vix = vix.rename(columns = {"^GSPC": "SP500", "^VIX": "VIX"})

In [ ]:
# create log returns for each column
for col in vix.columns:
    vix[col] = np.log(vix[col] / vix[col].shift(1))
    
vix = vix.dropna()

In [ ]:
# plot S&P500 to VIX on a scatter graph
plt.scatter(vix.SP500, vix.VIX)

plt.title("S&P500 to VIX Correlation (30 Year)")
plt.ylabel("S&P500")
plt.xlabel("VIX")

# create a trendline
z = np.polyfit(vix.SP500, vix.VIX, 1)
p = np.poly1d(z)
plt.plot(vix.SP500, p(vix.SP500), color = "r")

#plt.tight_layout()
#plt.savefig("SP500_VIX.png")

plt.show()

In [ ]:
# run tests of correlations
pear = sp.pearsonr(vix.SP500, vix.VIX)
spea = sp.spearmanr(vix.SP500, vix.VIX)

# place results into a data frame
data = {"Pearson": pear[0:2], "Spearman": spea[0:2]}
corr = pd.DataFrame(data, index = ["Correlation", "p-value"])
corr.round(3)